In [1]:
import numpy as np
import os
import pandas as pd #data handling library
import sklearn as sk

def get_RPKM_by_file(file_name, data_fields):
    df = pd.read_table(file_name, usecols=data_fields)
    df = df.T # tanspose
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df = df.rename(columns = new_header.T) #set the header row as the df header
    return df


def get_RPKM_by_filelist(list_fname):
    RPKM_list = []
    data_fields = ('gene','RPKM')
    with open(list_fname, 'r') as f:
        RPKM_list.extend(map(lambda data_file: get_RPKM_by_file(data_file, data_fields), f.readlines()))
    return RPKM_list

def get_RPKM_by_dir(data_dir):
    RPKM_list = []
    data_fields = ('gene','RPKM')
    for dirName, subdirList, fileList in os.walk(data_dir):
        RPKM_list.extend(map(lambda f: get_RPKM_by_file(os.path.join(dirName, f), data_fields), fileList))
    return RPKM_list

data_dir = './Data'
RPKM_list = get_RPKM_by_dir(data_dir)
print RPKM_list[0]

     ?|100130426 ?|100133144 ?|100134869   ?|10357   ?|10431 ?|136542  \
RPKM           0   0.7934492   0.4086566  3.628636  37.58069        0   

      ?|155060    ?|26823 ?|280660 ?|317712     ...     ZXDA|7789 ZXDB|158586  \
RPKM  3.639547  0.3250151        0        0     ...      1.292995    6.104444   

     ZXDC|79364 ZYG11A|440590 ZYG11B|79699  ZYX|7791 ZZEF1|23140 ZZZ3|26009  \
RPKM   11.99822     0.5127433     6.214898  119.5185    14.80452    10.5448   

     psiTPTE22|387590 tAKR|389932  
RPKM         13.92924   0.2304816  

[1 rows x 20532 columns]


In [2]:
def merge_data(data_list):
    df = pd.concat(data_list, axis=0, join='inner', join_axes=None, ignore_index=False,
        keys=None, levels=None, names=None, verify_integrity=False)
    return df

X = merge_data(RPKM_list)
Y = np.ravel(map(round, np.random.rand(len(X), 1)))

print X

     ?|100130426 ?|100133144 ?|100134869   ?|10357   ?|10431    ?|136542  \
RPKM           0   0.7934492   0.4086566  3.628636  37.58069           0   
RPKM  0.06694772   0.9270708   0.4969419  5.518766  22.36725           0   
RPKM           0    1.180236    0.450388  4.992169  24.97131           0   
RPKM  0.04956242   0.9206793   0.3488647  3.163578  14.72296           0   
RPKM           0   0.6563802   0.3193393  3.247601  42.65823  0.02245179   
RPKM           0     1.19084   0.4598564  4.235559  32.02006           0   
RPKM           0    1.363317    0.541452  4.209579   25.9165           0   
RPKM           0   0.4957196   0.2666096  5.330419  44.28414           0   
RPKM           0    1.180167   0.5171587  3.949443  18.97144           0   
RPKM           0   0.9261403   0.4104143  2.862239  39.80732           0   
RPKM           0   0.9073837   0.3937844  6.820181   45.4014           0   
RPKM           0    1.433736   0.5303374  2.528682  39.12532           0   
RPKM  0.0251

In [3]:
# Impute missing data, assuming X is all numeric values. Y can be strings.
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X_full = imp.fit_transform(X)

imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
Y_full = imp.fit_transform(Y)

In [4]:
# Encode classes as numbers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
nonnumeric_columns_X = []
nonnumeric_columns_Y = []
if nonnumeric_columns_X:
    for feature in nonnumeric_columns_X:
        X_full[feature] = le.fit_transform(X_full[feature])
if nonnumeric_columns_Y:
    for feature in nonnumeric_columns_Y:
        Y_full[feature] = le.fit_transform(Y_full[feature])

In [5]:
# Split data into train and test sets.
def split_data(X, Y, p_train):
    msk = np.random.rand(len(RPKM_list)) < p_train
    print Y[0]
    print msk
    X_train = X[msk]
    Y_train = Y[0][msk]
    return X[msk], X[~msk], Y[0][msk], Y[0][~msk]

train_X, test_X, train_Y, test_Y = split_data(X_full, Y_full, 0.6)

[ 1.  0.  1.  0.  1.  0.  0.  1.  1.  0.  1.  1.  1.  1.  1.  1.  0.]
[ True  True False  True  True  True  True False  True  True  True  True
 False  True  True  True  True]


In [6]:
from utility import statsTest
df_X_train = pd.DataFrame(train_X)
df_Y_train = pd.DataFrame(train_Y)
stats = statsTest.statistics_test(df_X_train, df_Y_train)
stats.index = X.columns.values
stats.columns = ['pvalue']# adjusted
stats = stats.sort(columns=['pvalue']) #adjusted
print stats[0:10]

                     pvalue
SRR|63826         39.950625
ANO10|55129       39.950625
DYNC1LI2|1783     39.950625
PSMD13|5719       39.950625
GSTTP2|653399     39.950625
LOC285375|285375  39.950625
SNHG11|128439     39.950625
NCRNA00105|80161  39.950625
CEMP1|752014      39.950625
SLC35A1|10559     39.950625


In [7]:
import xgboost as xgb
#gbm = xgb.XGBClassifier(max_depth=10, n_estimators=300, learning_rate=0.05).fit(train_X, train_Y)
gbm = xgb.XGBClassifier(max_depth=20, n_estimators=300).fit(train_X, train_Y)
predictions = gbm.predict(test_X)


ImportError: No module named xgboost

In [8]:
print test_Y - predictions

NameError: name 'predictions' is not defined

In [9]:
gbm.get_params()

NameError: name 'gbm' is not defined

In [10]:
tree = xgb.to_graphviz(gbm)
tree.render('xgboost_tree.gv', view=True)

NameError: name 'xgb' is not defined

In [11]:
print xgb.plot_importance(gbm)
import matplotlib.pyplot as plt
plt.savefig('xgboost_importance.png')

NameError: name 'xgb' is not defined

In [50]:
from sklearn.ensemble import gradient_boosting, GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, roc_curve
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from utility.KFoldCV import kfoldCV


def evaluate(r, d):
    tp = 0.
    fp = 0.
    tn = 0.
    fn = 0.
    for i in range(r.shape[0]):
        if r[i] == d[i]:
            if d[i] == 1:
                tp += 1
            else:
                fp += 1
        else:
            if d[i] == 1:
                fn +=1
            else:
                tn +=1
    k = np.array([tp, fp, tn, fn, (tp+tn)/r.shape[0]])
    return k


def clffit(t1, d1, t2, d2, clf):
    clf.fit(t1, d1)
    r = clf.predict(t2)
    return evaluate(r, d2)


def classify(data, labels):
    lrs = [0.1, 0.5, 1]
    estimateNum = [50, 100, 200]
    depths = [3, 4, 5]
    blr = 0
    be = 0
    bd = 0
    ba = -np.inf
    for lr in lrs:
        for en in estimateNum:
            for d in depths:
                clf = GradientBoostingClassifier(learning_rate=lr, n_estimators=en, max_depth=d, random_state=0)
                s = kfoldCV(data, labels, clffit, 5, clf)
                if s[-1] > ba:
                    ba = s[-1]
                    blr = lr
                    be = en
                    bd = d
    X = data.as_matrix()
    y = labels.as_matrix()
    y = y.reshape([y.shape[1]])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=0)

    clf = GradientBoostingClassifier(learning_rate=blr, n_estimators=be, max_depth=bd)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    au = accuracy_score(y_test, y_pred)
    fpr, tpr, t = roc_curve(y_test, y_pred)
    plt.plot(fpr, tpr, label='roc')
    plt.xlabel('false positive rate')
    plt.ylabel('true positive rate')
    plt.legend(loc=1)
    plt.savefig('roc.png')

    return au, clf.feature_importances_


In [ ]:
au, importance = classify(pd.DataFrame(X_full), pd.DataFrame(Y_full))
print au
print importance

In [48]:
k = np.where(importance!=0)[0]
for t in k:
        print t, importance[t]

462 0.01
600 0.01
1605 0.01
2761 0.01
4021 0.01
16843 0.01
18183 0.01


In [49]:
np.where(importance!=0)

(array([  462,   600,  1605,  2761,  4021, 16843, 18183]),)